In [1]:
import torch
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True,trust_repo=True)

Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/zipball/master" to C:\Users\arsha/.cache\torch\hub\master.zip
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/releases/download/v1.0/unet-e012d006.pt" to C:\Users\arsha/.cache\torch\hub\checkpoints\unet-e012d006.pt


In [2]:
import urllib
url, filename = ("https://github.com/mateuszbuda/brain-segmentation-pytorch/raw/master/assets/TCGA_CS_4944.png", "TCGA_CS_4944.png")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [3]:
import torchvision
from torchvision.transforms import transforms
import numpy as np
from PIL import Image
import torch


# Load the image
input_image = Image.open(filename)

# Convert the image to a NumPy array for calculating mean and std
input_image_np = np.array(input_image)

# Calculate mean and std (for each channel)
m, s = np.mean(input_image_np, axis=(0, 1)), np.std(input_image_np, axis=(0, 1))

# Preprocessing pipeline
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=m / 255.0, std=s / 255.0),  # Normalize expects values between 0 and 1
])

# Preprocess the image
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# Check if GPU is available and move data/model to GPU if so
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model = model.to('cuda')

# Inference
with torch.no_grad():
    output = model(input_batch)

# Print the rounded output
print(torch.round(output[0]))


tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


In [7]:
from typing import Optional

import requests
from batchgenerators.utilities.file_and_folder_operations import *
from time import time
from nnunetv2.model_sharing.model_import import install_model_from_zip_file
from nnunetv2.paths import nnUNet_results
from tqdm import tqdm

In [8]:
def download_and_install_from_url(url):
    assert nnUNet_results is not None, "Cannot install model because network_training_output_dir is not " \
                                                    "set (RESULTS_FOLDER missing as environment variable, see " \
                                                    "Installation instructions)"
    print('Downloading pretrained model from url:', url)
    import http.client
    http.client.HTTPConnection._http_vsn = 10
    http.client.HTTPConnection._http_vsn_str = 'HTTP/1.0'

    import os
    home = os.path.expanduser('~')
    random_number = int(time() * 1e7)
    tempfile = join(home, f'.nnunetdownload_{str(random_number)}')

    try:
        download_file(url=url, local_filename=tempfile, chunk_size=8192 * 16)
        print("Download finished. Extracting...")
        install_model_from_zip_file(tempfile)
        print("Done")
    except Exception as e:
        raise e
    finally:
        if isfile(tempfile):
            os.remove(tempfile)

In [9]:
def download_file(url: str, local_filename: str, chunk_size: Optional[int] = 8192 * 16) -> str:
    # borrowed from https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True, timeout=100) as r:
        r.raise_for_status()
        with tqdm.wrapattr(open(local_filename, 'wb'), "write", total=int(r.headers.get("Content-Length"))) as f:
            for chunk in r.iter_content(chunk_size=chunk_size):
                f.write(chunk)
    return local_filename

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()
        self.encoder1 = self.conv_block(in_channels, 64)
        self.encoder2 = self.conv_block(64, 128)
        self.encoder3 = self.conv_block(128, 256)
        self.encoder4 = self.conv_block(256, 512)
        self.bottleneck = self.conv_block(512, 1024)
        
        self.decoder4 = self.upconv_block(512, 512)
        self.decoder3 = self.upconv_block(512, 256)
        self.decoder2 = self.upconv_block(256, 128)
        self.decoder1 = self.upconv_block(128, 64)
        
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Debug print statements to trace the tensor shapes
        print(f"Input shape: {x.shape}")
        
        enc1 = self.encoder1(x)
        print(f"After encoder1: {enc1.shape}")
        
        enc2 = self.encoder2(F.max_pool2d(enc1, kernel_size=2))
        print(f"After encoder2: {enc2.shape}")
        
        enc3 = self.encoder3(F.max_pool2d(enc2, kernel_size=2))
        print(f"After encoder3: {enc3.shape}")
        
        enc4 = self.encoder4(F.max_pool2d(enc3, kernel_size=2))
        print(f"After encoder4: {enc4.shape}")
        
        bottleneck = self.bottleneck(F.max_pool2d(enc4, kernel_size=2))
        print(f"After bottleneck: {bottleneck.shape}")
        
        dec4 = self.decoder4(bottleneck)
        print(f"After decoder4 (before concat): {dec4.shape}")
        dec4 = torch.cat((dec4, enc4), dim=1)
        print(f"After decoder4 (after concat): {dec4.shape}")
        
        dec3 = self.decoder3(dec4)
        print(f"After decoder3 (before concat): {dec3.shape}")
        dec3 = torch.cat((dec3, enc3), dim=1)
        print(f"After decoder3 (after concat): {dec3.shape}")
        
        dec2 = self.decoder2(dec3)
        print(f"After decoder2 (before concat): {dec2.shape}")
        dec2 = torch.cat((dec2, enc2), dim=1)
        print(f"After decoder2 (after concat): {dec2.shape}")
        
        dec1 = self.decoder1(dec2)
        print(f"After decoder1 (before concat): {dec1.shape}")
        dec1 = torch.cat((dec1, enc1), dim=1)
        print(f"After decoder1 (after concat): {dec1.shape}")
        
        output = self.final_conv(dec1)
        print(f"Output shape: {output.shape}")
        return output

# Initialize the model and load the weights
model = UNet(in_channels=3, out_channels=1)  # Adjust channels as needed
model.load_state_dict(torch.load('3channel50.pt', map_location=torch.device('cpu'),weights_only=True), strict=False)

# Set the model to evaluation mode
model.eval()

FileNotFoundError: [Errno 2] No such file or directory: '3channel50.pt'

In [50]:
import pydicom
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image

def preprocess_dicom_image(image_path):
    # Load DICOM image using pydicom
    dcm_data = pydicom.dcmread(image_path)

    # Extract pixel data (might require reshaping depending on your model)
    image_data = dcm_data.pixel_array

    # Convert to RGB (if necessary, depending on your model)
    image_data = Image.fromarray(image_data).convert('RGB')  # Convert to PIL Image for consistency
    
    # Resize the image if necessary
    image_data = image_data.resize((256, 256))

    # Convert to tensor
    image_tensor = transforms.ToTensor()(image_data)
   

    # Add batch dimension
    image_tensor = image_tensor.unsqueeze(0)  # Shape: (1, 3, 256, 256)
    print(image_tensor.shape)
    return image_tensor

def postprocess_output(output_tensor):
    """Postprocesses the output tensor from a segmentation model.

    Args:
        output_tensor (torch.Tensor): Output tensor from the model.

    Returns:
        np.ndarray: Segmented image as a NumPy array.
    """

    # Convert tensor to NumPy array
    output_array = output_tensor.squeeze(0).detach().cpu().numpy()  # Shape: (1, 256, 256)

    # Binarize the output (thresholding)
    output_array = (output_array > 0.5).astype(np.uint8)  # Adjust threshold as necessary

    return output_array * 255  # Scale to [0, 255] for visualization

# Load and preprocess the DICOM image
input_image_path = r'C:\Users\sujal\PycharmProjects\DSU\dicom_communication\0.dcm'  # Replace with your DICOM image path
input_tensor = preprocess_dicom_image(input_image_path)

# Perform inference
with torch.no_grad():  # Disable gradient calculation
    output_tensor = model(input_tensor)

# Postprocess the output
segmented_image = postprocess_output(output_tensor)

# Display or save the segmented image
segmented_image_pil = Image.fromarray(segmented_image[0])  # Convert to PIL Image
segmented_image_pil.show()  # Show the image

torch.Size([1, 3, 256, 256])
Input shape: torch.Size([1, 3, 256, 256])
After encoder1: torch.Size([1, 64, 256, 256])
After encoder2: torch.Size([1, 128, 128, 128])
After encoder3: torch.Size([1, 256, 64, 64])
After encoder4: torch.Size([1, 512, 32, 32])
After bottleneck: torch.Size([1, 1024, 16, 16])


RuntimeError: Given transposed=1, weight of size [512, 512, 2, 2], expected input[1, 1024, 16, 16] to have 512 channels, but got 1024 channels instead